### Yolov3 with Vitis AI Tools
This lab demonstrates the Vitis AI workflow on TensorFlow for a Yolov3 network trained on the Pascal VOC data set. Ths includes network freeze, quantization, evaluation, compilation and deployment on the ZCU104 Board

### Import Necessary Python Packages

In [ ]:
!pip install Pillow --user
!pip install tqdm --user --no-warn-script-location

### Yolov3 VOC Model 

The YoloV3 model we are using model was trained on the Pascal VOC data set, and is availabe on the Xilinx AI Model Zoo.
The input image size is 416x416. The model has already been frozen.
Visit https://github.com/Xilinx/Vitis-AI/tree/master/AI-Model-Zoo for more details.

![title](pictures/vitis_ai_model_zoo.PNG)

## Download TenfsorFlow Yolov3 VOC model from ModelZoo
If you are running the Lab on the AWS AMI you can skip this step. The model has already been download for you.
If you are running on your own machine uncomment the lines below to run

In [ ]:
#!wget -O tf_yolov3_voc_416_416_1.0.zip  https://www.xilinx.com/bin/public/openDownload?filename=tf_yolov3_voc_416_416_1.0.zip 
#!unzip -j -d model_data tf_yolov3_voc_416_416_1.0.zip tf_yolov3_voc_416_416_65.63G/float/yolov3_voc.pb
#!unzip -j -d model_data tf_yolov3_voc_416_416_1.0.zip tf_yolov3_voc_416_416_65.63G/test_code/voc_data/*.txt
#!rm tf_yolov3_voc_416_416_1.0.zip

## DownLoad 2007 Pascal VOC Test Images
We will be using the first 100 jpeg images in the 2007 Pascal VOC Data Set.
If you are running the Lab on the AWS AMI you can skip this step. The files have already been downloaded for you.
If you are running on your own machine uncomment the lines below to run

#### Pascal 2007 VOC Citation:
@misc{pascal-voc-2007,
	author = "Everingham, M. and Van~Gool, L. and Williams, C. K. I. and Winn, J. and Zisserman, A.",
	title = "The {PASCAL} {V}isual {O}bject {C}lasses {C}hallenge 2007 {(VOC2007)} {R}esults",
	howpublished = "http://www.pascal-network.org/challenges/VOC/voc2007/workshop/index.html"}	

In [ ]:
#!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtest_06-Nov-2007.tar
#!tar -xvf VOCtest_06-Nov-2007.tar
#!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar
#!tar -xvf VOCtrainval_06-Nov-2007.tar
#!mkdir VOC2007
#!mkdir VOC2007/JPEGImages
#!cp VOCdevkit/VOC2007/JPEGImages/*.jpg VOC2007/JPEGImages/
#!rm -rf VOCdevkit
#!rm -f *.tar
#!ln -s VOC2007/JPEGImages images



## VAI TensorFlow Quantization Tool (DECENT)
### Inspect Networkt
Use the "inspect" function in to check input nodes and output nodes of the network. We will use this information in later steps. 

In [ ]:
!vai_q_tensorflow inspect --input_frozen_graph=model_data/yolov3_voc.pb 

### Generate predictions with Floating Point Network
The following python file will run inferrence by reading the images listed in the file: voc_data/2007_test.txt.

In [ ]:
!python ./core/tf_prediction.py --pb_file model_data/yolov3_voc.pb      \
                                --test_list "model_data/2007_test.txt"    \
                                --result_file "model_data/2007_test_pred.txt"



The file voc_data/2007_test_pred.txt will be created which lists the detected class, score and box locations for all objects.


![title](pictures/test_results.PNG)

### Evalute Floating Point  Network
This python function will calculate Mean Average Precision (mAP) accuracy for the network using the images from the test data set.

In [ ]:
!python ./core/evaluation.py -mode detection \
                             -detection_use_07_metric True \
                             -gt_file ./model_data/2007_test_gt.txt \
                             -result_file ./model_data/2007_test_pred.txt \
                             -detection_iou 0.5 \
                             -detection_thresh 0.005

When evaluation is complete, you will see that the mAP for the images in the test data set is 0.7858

### Quantize the Network
The VAI quantizer (decnt) will now convert the floating point weights and activations to INT8. 
A small subset of the training data (typicall 100 images) is needed for quantization calibration. These are input into the the quantizer using the python based input function: input_fn.py (show below):

![title](pictures/input_fn.PNG)

The input function reads the calibration image and performs letterboxing (416x416 resize with aspect ratio preservation).

This will take a few minutes to run on a GPU enabled machine, and much longer on a CPU only machine.




In [ ]:
!vai_q_tensorflow quantize \
  --input_frozen_graph model_data/yolov3_voc.pb \
  --input_nodes input_1 \
  --input_shapes ?,416,416,3 \
  --output_nodes conv2d_59/BiasAdd,conv2d_67/BiasAdd,conv2d_75/BiasAdd \
  --input_fn input_fn.calib_input \
  --method 1 \
  --gpu 0 \
  --calib_iter 100 


### Generate predictions with Quantized  Network
Similar to what we did with the floating point model, we will now run inference and generate predictions using the quantized model.

In [ ]:
!python ./core/tf_prediction.py --pb_file quantize_results/quantize_eval_model.pb      \
                                --test_list "model_data/2007_test.txt"    \
                                --result_file "model_data/2007_test_quantize.txt"

### Evalute the Quantized Network
Now we will evaluate the Quantized network, and measure accuarcy after quantization

In [ ]:
!python ./core/evaluation.py -mode detection \
                             -detection_use_07_metric True \
                             -gt_file ./model_data/2007_test_gt.txt \
                             -result_file ./model_data/2007_test_quantize.txt \
                             -detection_iou 0.5 \
                             -detection_thresh 0.005

The mAP after quantization is 0.7744 as you can see there only a small accuracy loss converting from Float32 to INT8. 

## VAI Compilation Tool 
### Compile Network
We will now compile the quantizied network. This will generate the instruction set for the Xilinx Deep Learning Processor Unit (DPU). The DPU is a configurable computation engine dedicated for convolutional neural networks. The --arch file point to a .json file which specifes the DPU configuration. In our case we are targetting the DPU config for the ZCU104 board.

In [1]:
!vai_c_tensorflow --frozen_pb quantize_results/deploy_model.pb --arch /opt/vitis_ai/compiler/arch/DPUCZDX8G/ZCU104/arch.json --output_dir vai_c_output --net_name yolov3_voc --options "{'cpu_arch':'arm64', 'mode':'normal', 'save_kernel':''}"
!ls -l vai_c_output



**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************

Kernel topology "yolov3_voc_kernel_graph.jpg" for network "yolov3_voc"
kernel list info for network "yolov3_voc"
                               Kernel ID : Name
                                       0 : yolov3_voc

                             Kernel Name : yolov3_voc
--------------------------------------------------------------------------------
                             Kernel Type : DPUKernel
                               Code Size : 0.93MB
                              Param Size : 58.75MB
                           Workload MACs : 65427.98MOPS
                         IO Memory Space : 8.43MB
                              Mean Value : 0, 0, 0, 
                      Total Tensor Count : 80
                Boundary Input Tensor(s)   (H*W*C)
                            input_1:0(0) : 416*416*3

               Boundary Output Tensor(s)   (H

## Final Instructions
After compilation, the file dpu_yolov3_voc.elf is generated. This part of the lab completed, please go back to the terminal and enter CtrC to close the Jupyter Notebook server and then CtrlD to close the docker shell. You can also close this and any Jupyter notebooks you currently have open in your browser.